## Final Project 
## Name:  Andrew, Lyla, Trina

In [1]:
import numpy as np
from PIL import Image
from scipy.spatial import distance
from scipy.optimize import linear_sum_assignment
from matplotlib import pyplot as plt
from munkres import Munkres
from sklearn.cluster import KMeans

In [2]:
#processing the output file

# This is the baseline code for merging the fragment tracklet using clustering based on appearance


class postprocess:
    def __init__(self,number_of_people,cluster_method):
        self.n = number_of_people
        if cluster_method == 'kmeans':
            self.cluster_method = KMeans(n_clusters=self.n, random_state=0)
        else:
            raise NotImplementedError
    
    def run(self,features):

        print('Start Clustering')

        self.cluster_method.fit(features)

        print('Finish Clustering')

        return self.cluster_method.labels_

In [3]:
#IOU based clustering

# calculate the overlap ratio of two bounding boxes
def calculate_iou(bbox1, bbox2):
    x1_1, y1_1, x2_1, y2_1 = bbox1
    x1_2, y1_2, x2_2, y2_2 = bbox2
    x_left = max(x1_1, x1_2)
    y_top = max(y1_1, y1_2)
    x_right = min(x2_1, x2_2)
    y_bottom = min(y2_1, y2_2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    area_bbox1 = (x2_1 - x1_1 + 1) * (y2_1 - y1_1 + 1)
    area_bbox2 = (x2_2 - x1_2 + 1) * (y2_2 - y1_2 + 1)
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)

    iou = intersection_area / float(area_bbox1 + area_bbox2 - intersection_area)
    # Calculate additional similarity metrics
    center1_x = (x1_1 + x2_1) / 2
    center1_y = (y1_1 + y2_1) / 2
    center2_x = (x1_2 + x2_2) / 2
    center2_y = (y1_2 + y2_2) / 2
    distance = np.sqrt((center1_x - center2_x)**2 + (center1_y - center2_y)**2)

    # Combine the similarity metrics (IoU and distance) into a single similarity score
    similarity_score = 0.7 * iou + 0.3 * (1 / (1 + distance))  # Example combination

    return similarity_score
    #return iou


# base class for tracklet
class Tracklet:
    def __init__(self, tracking_ID, box, feature, time, confidence):
        self.ID = tracking_ID
        self.boxes = [box]
        self.features = [feature]
        self.times = [time]
        self.confidences = [confidence]

        self.cur_box = box
        self.cur_feature = None
        self.alive = True

        self.final_features = None

    def update(self, box, feature, time, confidence):
        self.cur_box = box
        self.boxes.append(box)
        self.cur_feature = None
        self.features.append(feature)
        self.times.append(time)
        self.confidences.append(confidence)
        self.get_avg_features()

    def close(self):
        self.alive = False

    def get_avg_features(self):
        self.final_features = (
            sum(self.features) / len(self.features)
            if self.features
            else None
        )


# class for multi-object tracker
class tracker:
    def __init__(self):
        self.all_tracklets = []
        self.cur_tracklets = []

    def run(self, detections, features):
        for frame_id in range(0, 18010):
            if frame_id % 1000 == 0:
                print('Tracking | cur_frame {} | total frame 18010'.format(frame_id))

            inds = detections[:, 1] == frame_id
            cur_frame_detection = detections[inds]
            cur_frame_features = features[inds]

            # no tracklets in the first frame
            if len(self.cur_tracklets) == 0:
                for idx in range(len(cur_frame_detection)):
                    new_tracklet = Tracklet(
                        len(self.all_tracklets) + 1,
                        cur_frame_detection[idx][3:7],
                        cur_frame_features[idx],
                        frame_id,
                        cur_frame_detection[idx][2]  # confidence value
                    )
                    self.cur_tracklets.append(new_tracklet)
                    self.all_tracklets.append(new_tracklet)

            else:
                cost_matrix = np.zeros((len(self.cur_tracklets), len(cur_frame_detection)))

                for i in range(len(self.cur_tracklets)):
                    for j in range(len(cur_frame_detection)):
                        cost_matrix[i][j] = 1 - calculate_iou(
                            self.cur_tracklets[i].cur_box, cur_frame_detection[j][3:7]
                        )

                row_inds, col_inds = linear_sum_assignment(cost_matrix)

                matches = min(len(row_inds), len(col_inds))

                for idx in range(matches):
                    row, col = row_inds[idx], col_inds[idx]
                    if cost_matrix[row, col] == 1:
                        self.cur_tracklets[row].close()
                        new_tracklet = Tracklet(
                            len(self.all_tracklets) + 1,
                            cur_frame_detection[col][3:7],
                            cur_frame_features[col],
                            frame_id,
                            cur_frame_detection[col][2]  # confidence value
                        )
                        self.cur_tracklets.append(new_tracklet)
                        self.all_tracklets.append(new_tracklet)
                    else:
                        self.cur_tracklets[row].update(
                            cur_frame_detection[col][3:7],
                            cur_frame_features[col],
                            frame_id,
                            cur_frame_detection[col][2],  # confidence value
                        )

                # initiate unmatched detections as new tracklets
                for idx, det in enumerate(cur_frame_detection):
                    if idx not in col_inds:
                        new_tracklet = Tracklet(
                            len(self.all_tracklets) + 1,
                            det[3:7],
                            cur_frame_features[idx],
                            frame_id,
                            det[2],  # confidence value
                        )
                        self.cur_tracklets.append(new_tracklet)
                        self.all_tracklets.append(new_tracklet)

            self.cur_tracklets = [trk for trk in self.cur_tracklets if trk.alive]

        final_tracklets = self.all_tracklets

        for trk_id in range(len(final_tracklets)):
            final_tracklets[trk_id].get_avg_features()

        return final_tracklets


In [11]:

if __name__ == "__main__":

    #camera = 74 # validation set
    camera = 75  # test set

    # The number of people in the dataset is 5. Bonus points for method that does not require this line of hard coding
    number_of_people = 5


    result_path = 'baseline_result.txt'

    # Load the data
    detection = np.loadtxt('./detection.txt', delimiter=',', dtype=None)
    embedding = np.load('./embedding.npy', allow_pickle=True)
    inds = detection[:, 0] == camera
    test_detection = detection[inds]
    test_embedding = embedding[inds]
    sort_inds = test_detection[:, 1].argsort()
    test_detection = test_detection[sort_inds]
    test_embedding = test_embedding[sort_inds]

    # Filter out low-confidence detections

    confidence_threshold = 0.5
    confidence_mask = test_detection[:, 7] >= confidence_threshold
    test_detection = test_detection[confidence_mask]
    test_embedding = test_embedding[confidence_mask]


    mot = tracker()
    postprocessing = postprocess(number_of_people, 'kmeans')

    # Run the IoU tracking
    tracklets = mot.run(test_detection, test_embedding)

    features = np.array([trk.final_features for trk in tracklets])

    # Run the Post Processing to merge the tracklets
    labels = postprocessing.run(features)  # The label represents the final tracking ID, it starts from 0. We will make it start from 1 later.

    tracking_result = []

    print('Writing Result ...')

    for i, trk in enumerate(tracklets):
        final_tracking_id = labels[i] + 1  # make it start with 1
        for idx in range(len(trk.boxes)):
            frame = trk.times[idx]
            x1, y1, x2, y2 = trk.boxes[idx]
            x, y, w, h = x1, y1, x2 - x1, y2 - y1

            result = '{},{},{},{},{},{},{},-1,-1 \n'.format(camera, final_tracking_id, frame, x, y, w, h)

            tracking_result.append(result)

    print('Save tracking results at {}'.format(result_path))

    with open(result_path, 'w') as f:
        f.writelines(tracking_result)


Tracking | cur_frame 0 | total frame 18010
Tracking | cur_frame 1000 | total frame 18010
Tracking | cur_frame 2000 | total frame 18010
Tracking | cur_frame 3000 | total frame 18010
Tracking | cur_frame 4000 | total frame 18010
Tracking | cur_frame 5000 | total frame 18010
Tracking | cur_frame 6000 | total frame 18010
Tracking | cur_frame 7000 | total frame 18010
Tracking | cur_frame 8000 | total frame 18010
Tracking | cur_frame 9000 | total frame 18010
Tracking | cur_frame 10000 | total frame 18010
Tracking | cur_frame 11000 | total frame 18010
Tracking | cur_frame 12000 | total frame 18010
Tracking | cur_frame 13000 | total frame 18010
Tracking | cur_frame 14000 | total frame 18010
Tracking | cur_frame 15000 | total frame 18010
Tracking | cur_frame 16000 | total frame 18010
Tracking | cur_frame 17000 | total frame 18010


/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f4bbb43fec0>
Traceback (most recent call last):
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
  

Tracking | cur_frame 18000 | total frame 18010
Start Clustering


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f4bbb43ff60>
Traceback (most recent call last):
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/home/andrew/miniconda3/envs/cuda/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
A

Finish Clustering
Writing Result ...
Save tracking results at baseline_result.txt


In [12]:
#create the images for visualization

!python ./visualization.py ./baseline_result.txt --dstype=test

Saving visualization result at test_visualization
processing frame:  0
processing frame:  100
processing frame:  200
processing frame:  300
processing frame:  400
processing frame:  500
processing frame:  600
processing frame:  700
processing frame:  800
processing frame:  900
processing frame:  1000


In [13]:
#visualize the images

from pathlib import Path
import imageio

image_path = Path('./test_visualization')
images = list(image_path.glob('*.jpg'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    


/tmp/ipykernel_9689/2908565192.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_list.append(imageio.imread(file_name))


In [14]:
# create a gif
imageio.mimwrite('./checkcheck2.gif', image_list, fps=3)

In [ ]:
#evaluate the result
!python ./evaluation/eval.py ./baseline_result.txt ./evaluation/gt/74.txt

          IDF1   IDP   IDR    idtp   idfp   idfn  MOTA  MOTP  Rcll  Prcn  IDs MT PT ML
MultiCam 86.80 81.41 86.67 27262.0 6226.0 4194.0 85.04 10.67 95.03 95.32 1674  3  1  1
